In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [140]:
data=pd.read_excel(r'training.xlsx')
data.head()


,Date,Vehicle Category,GVWR Class,Fuel Type,Model Year,Fuel Technology,Electric Mile Range,Number of Vehicles Registered at the Same Address,Region,Vehicle Population
0,2020,BS,Not Applicable,Gasoline,2023.0,ICE,Not Applicable,1,Statewide,1
1,2019,T3,Unknown,Gasoline,2022.0,ICE,Not Applicable,2,Statewide,1
2,2020,MH,Not Applicable,Gasoline,2023.0,ICE,Not Applicable,2,Statewide,1
3,2019,BS,Not Applicable,Diesel,2022.0,ICE,Not Applicable,≥4,Statewide,26
4,2019,MH,Not Applicable,Gasoline,2022.0,ICE,Not Applicable,≥4,Statewide,55


In [141]:
def clean(data):
    df = pd.DataFrame(data)
    df.columns = df.columns.str.strip().str.lower().str.replace(r'\W+', '_', regex=True)
    df.drop(columns=['region'], inplace=True)
    df['model_year'] = df.groupby(['fuel_type', 'gvwr_class'])['model_year'].transform(lambda x: x.fillna(x.median())).astype(int)
    df['age'] = df['date']-df['model_year']
    return df

In [142]:
df = clean(data)
df.head()

,date,vehicle_category,gvwr_class,fuel_type,model_year,fuel_technology,electric_mile_range,number_of_vehicles_registered_at_the_same_address,vehicle_population,age
0,2020,BS,Not Applicable,Gasoline,2023,ICE,Not Applicable,1,1,-3
1,2019,T3,Unknown,Gasoline,2022,ICE,Not Applicable,2,1,-3
2,2020,MH,Not Applicable,Gasoline,2023,ICE,Not Applicable,2,1,-3
3,2019,BS,Not Applicable,Diesel,2022,ICE,Not Applicable,≥4,26,-3
4,2019,MH,Not Applicable,Gasoline,2022,ICE,Not Applicable,≥4,55,-3


In [143]:
X = df.drop(columns=['vehicle_population'])  
y = df['vehicle_population']  


In [144]:
scoring_data = pd.read_excel(r'scoring.xlsx')
scoring_data = clean(scoring_data)

In [145]:
categorical_features = ['fuel_technology', 'fuel_type','model_year', 'electric_mile_range','date', 'vehicle_category','gvwr_class','number_of_vehicles_registered_at_the_same_address']
numeric_features = ['age']
X[categorical_features] = X[categorical_features].astype(str)
X[numeric_features] = X[numeric_features].astype(int)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),  # keep numeric features
         ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # encode categorical features
    ]
)

# 4. decision tree regressor
model = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('regressor', DecisionTreeRegressor(random_state=42))  
])

# 5.split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, y_train = X,y


In [146]:
X_train

,date,vehicle_category,gvwr_class,fuel_type,model_year,fuel_technology,electric_mile_range,number_of_vehicles_registered_at_the_same_address,age
35746,2019,T3,2,Diesel,1981,ICE,Not Applicable,1,38
32643,2021,T3,2,Gasoline,1988,ICE,Not Applicable,3,33
21317,2021,P,Not Applicable,Gasoline,2002,ICE,Not Applicable,2,19
22883,2022,MC,Not Applicable,Diesel,2001,ICE,Not Applicable,1,21
12902,2020,MC,Not Applicable,Gasoline,2010,ICE,Not Applicable,3,10
...,...,...,...,...,...,...,...,...,...
6265,2023,T6,6,Diesel,2020,ICE,Not Applicable,≥4,3
11284,2021,T4,2,Gasoline,2013,ICE,Not Applicable,≥4,8
38158,2020,T1,1,Diesel,1979,ICE,Not Applicable,≥4,41
860,2022,T7,8,Natural Gas,2023,ICE,Not Applicable,1,-1


In [147]:
y_train

35746        1
32643      257
21317    45906
22883        1
12902     8585
         ...  
6265      3540
11284     9634
38158        2
860         25
15795        1
Name: vehicle_population, Length: 32842, dtype: int64

In [148]:

# 6. 
model.fit(X_train, y_train)

# 7. 
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE on test set: {rmse}')

# 8. 

scoring_df = pd.DataFrame(scoring_data)
scoring_df[categorical_features] = scoring_df[categorical_features].astype(str)
# 9. 
predictions = model.predict(scoring_df)

# 10. 
submission_data = {
    'Predictions': predictions 
}
submission_df = pd.DataFrame(submission_data)

# 
# submission_df.to_excel('submission_file_.xlsx', index=False, sheet_name='Predictions')

print("Predictions saved to submission_file.xlsx")

RMSE on test set: 4102.662843905157
Predictions saved to submission_file.xlsx


In [149]:
baseline_pred = np.full_like(y_test, y_train.mean())
baseline_rmse = root_mean_squared_error(y_test, baseline_pred)
print(f'Baseline RMSE: {baseline_rmse}')

# compare the model performance with the baseline model
if rmse < baseline_rmse:
    print("the model performance is better than the baseline model.")
else:
    print("the model performance is worse than the baseline model.")

Baseline RMSE: 20342.63507166346
the model performance is better than the baseline model.


- model performance using scoring.csv

In [150]:
import pandas as pd
import numpy as np
from sklearn.metrics import root_mean_squared_error

# 1. 
submission_df = pd.read_excel('submission_file_fill_na.xlsx', sheet_name='Predictions')
predictions = submission_df['Predictions']  

# 2. 
scoring_df = pd.read_excel('scoring.xlsx')
scoring_df=clean(scoring_df)
true_values = scoring_df['vehicle_population']  

# 3. RMSE
rmse = root_mean_squared_error(true_values, predictions)
print(f'RMSE: {rmse:.2f}')

RMSE: 5213.35


# R2 score


In [151]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f'R^2 on test set: {r2:.2f}')

R^2 on test set: 0.96


# Feature Importance

In [152]:
import numpy as np
import pandas as pd


onehot_feature_names = list(model.named_steps['preprocessor']
                                       .named_transformers_['cat']
                                       .get_feature_names_out(categorical_features))

feature_names = numeric_features + onehot_feature_names

importances = model.named_steps['regressor'].feature_importances_

importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

importance_df['Original Feature'] = importance_df['Feature'].apply(lambda x: x.split('_')[0] if '_' in x else x)
importance_summary = importance_df.groupby('Original Feature')['Importance'].sum().reset_index()

importance_summary = importance_summary.sort_values(by='Importance', ascending=False)

print(importance_summary)


  Original Feature  Importance
3             fuel    0.446360
0              age    0.322951
7          vehicle    0.100685
6           number    0.060293
5            model    0.024398
4             gvwr    0.022206
1             date    0.017025
2         electric    0.006082


# Random forest v.s. gradient boost
  

In [153]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# the random forest model
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),  
    ('regressor', RandomForestRegressor(random_state=42))  
])

# the gradient boosting model
gb_model = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('regressor', GradientBoostingRegressor(random_state=42))  
])

# 5. split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. train and evaluate the random forest model
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)
rf_rmse = root_mean_squared_error(y_test, rf_y_pred)
print(f'random forest RMSE: {rf_rmse:.2f}')

# 7. train and evaluate the gradient boosting model
gb_model.fit(X_train, y_train)
gb_y_pred = gb_model.predict(X_test)
gb_rmse = root_mean_squared_error(y_test, gb_y_pred)
print(f'gb RMSE: {gb_rmse:.2f}')

# 8. load（scoring.csv）

scoring_df[categorical_features] = scoring_df[categorical_features].astype(str)

# 9. use the model with lower RMSE
if rf_rmse < gb_rmse:
    best_model = rf_model
    print("choose random forest model for prediction.")
else:
    best_model = gb_model
    print("choose gradient boosting model for prediction.")

predictions = best_model.predict(scoring_df)

# 10. save to submission_file.xlsx
submission_data_ = {
    'Predictions': predictions  
}
submission_df = pd.DataFrame(submission_data_)

# submission_df.to_excel('submission_file_updated_fill_na.xlsx', index=False, sheet_name='Predictions')
# submission_df.to_excel('submission_file_updated_fill_na111.xlsx', index=False, sheet_name='Predictions')
print("Predictions saved to submission_file_updated.xlsx")

random forest RMSE: 3478.16
gb RMSE: 11223.15
choose random forest model for prediction.
Predictions saved to submission_file_updated.xlsx


In [154]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, rf_y_pred)
print(f'R^2 on test set: {r2:.2f}')

R^2 on test set: 0.97


#  evaluation using scoring.csv

In [155]:
import pandas as pd
import numpy as np
from sklearn.metrics import root_mean_squared_error


submission_df = pd.read_excel('submission_file_updated_fill_na.xlsx', sheet_name='Predictions')
predictions = submission_df['Predictions']  


scoring_df = pd.read_excel('scoring.xlsx')
true_values = scoring_df['Vehicle Population']  


rmse = root_mean_squared_error(true_values, predictions)
print(f'RMSE: {rmse:.2f}')

RMSE: 4773.67


# feature importance

In [156]:
import numpy as np
import pandas as pd


onehot_feature_names = list(rf_model.named_steps['preprocessor']
                                       .named_transformers_['cat']
                                       .get_feature_names_out(categorical_features))

feature_names = numeric_features + onehot_feature_names

importances = rf_model.named_steps['regressor'].feature_importances_
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

importance_df['Original Feature'] = importance_df['Feature'].apply(lambda x: x.split('_')[0] if '_' in x else x)
importance_summary = importance_df.groupby('Original Feature')['Importance'].sum().reset_index()

importance_summary = importance_summary.sort_values(by='Importance', ascending=False)
print(importance_summary)


  Original Feature  Importance
3             fuel    0.339229
0              age    0.314436
2         electric    0.122063
7          vehicle    0.100653
6           number    0.058836
5            model    0.026141
4             gvwr    0.022128
1             date    0.016514
